In [1]:
import pandas as pd
import warnings
#warnings.filterwarnings("ignore")

In [2]:
X_balance_scale = pd.read_csv('data/X_balance_scale', delimiter=',')
y_balance_scale = pd.read_csv('data/y_balance_Scale', delimiter=',')

X_car_eval = pd.read_csv('data/X_car_eval', delimiter=',')
y_car_eval = pd.read_csv('data/y_car_eval', delimiter=',')

X_contraceptive = pd.read_csv('data/X_contraceptive', delimiter=',')
y_contraceptive = pd.read_csv('data/y_contraceptive', delimiter=',')

X_dermatology = pd.read_csv('data/X_dermatology', delimiter=',')
y_dermatology = pd.read_csv('data/y_dermatology', delimiter=',')

X_glass = pd.read_csv('data/X_glass', delimiter=',')
y_glass = pd.read_csv('data/y_glass', delimiter=',')

X_hayes = pd.read_csv('data/X_hayes', delimiter=',')
y_hayes = pd.read_csv('data/y_hayes', delimiter=',')

X_heart = pd.read_csv('data/X_heart', delimiter=',')
y_heart = pd.read_csv('data/y_heart', delimiter=',')

X_new_thyroid = pd.read_csv('data/X_new_thyroid', delimiter=',')
y_new_thyroid = pd.read_csv('data/y_new_thyroid', delimiter=',')

X_page = pd.read_csv('data/X_page', delimiter=',')
y_page = pd.read_csv('data/y_page', delimiter=',')

X_pen_based = pd.read_csv('data/X_pen_based', delimiter=',')
y_pen_based = pd.read_csv('data/y_pen_based', delimiter=',')

X_shuttle = pd.read_csv('data/X_shuttle', delimiter=',')
y_shuttle = pd.read_csv('data/y_shuttle', delimiter=',')

X_vertebra = pd.read_csv('data/X_vertebra', delimiter=',')
y_vertebra = pd.read_csv('data/y_vertebra', delimiter=',')

X_wine = pd.read_csv('data/X_wine', delimiter=',')
y_wine = pd.read_csv('data/y_wine', delimiter=',')

X_yeast = pd.read_csv('data/X_yeast', delimiter=',')
y_yeast = pd.read_csv('data/y_yeast', delimiter=',')

X_fars = pd.read_csv('data/X_fars', delimiter=',')
y_fars = pd.read_csv('data/y_fars', delimiter=',')

In [3]:
X_list = [
    X_balance_scale, X_car_eval, X_contraceptive,
    X_dermatology, X_glass, X_hayes, X_heart,
    X_new_thyroid, X_page, X_pen_based, X_shuttle,
    X_vertebra, X_wine, X_yeast, X_fars
]

y_list = [
    y_balance_scale, y_car_eval, y_contraceptive,
    y_dermatology, y_glass, y_hayes, y_heart,
    y_new_thyroid, y_page, y_pen_based, y_shuttle,
    y_vertebra, y_wine, y_yeast, y_fars
]

name_list = [
    'Balance Scale', 'Car Evaluation', 'Contraceptive',
    'Dermatology', 'Glass', 'Hayes', 'Heart',
    'New Thyroid', 'Page', 'Pen Based', 'Shuttle',
    'Vertebra', 'Wine', 'Yeast', 'FARS'
]

# Prediction

In [4]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.ensemble import RUSBoostClassifier

from utils import *
from tqdm import tqdm

In [7]:
clf_dict = {
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'AdaBoost_SAMME': AdaBoostClassifier(n_estimators=100, algorithm='SAMME', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.3, max_features=2, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(verbose=0), 
    'RUSBoostClassifier': RUSBoostClassifier(n_estimators=200, random_state=42)
}

clf_score_dict = {}

In [6]:
from adacost import *

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X_car_eval, y_car_eval, test_size=0.2, random_state=42)

clf = AdaCost(algorithm = "SAMME.R",random_state = 100)

# Fit AdaCost
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)
probabilities = clf.predict_proba(X_test)  # If you need probabilities

# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

# Print the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

TypeError: __init__() got an unexpected keyword argument 'estimator'

In [1]:
import nltk
import sklearn
import numpy 

#print('The nltk version is {}.'.format(nltk.__version__))
print('The numpy version is {}.'.format(numpy.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

# The nltk version is 3.0.0.
# The scikit-learn version is 0.15.2.

c:\Users\aikyn\miniconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The numpy version is 1.26.4.
The scikit-learn version is 1.4.1.post1.


In [63]:
for clf_name, clf in tqdm(clf_dict.items()):
    score = {}
    
    for i, (X, y) in enumerate(zip(X_list, y_list)):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        clf.fit(X_train, y_train) 
        
        y_preds = clf.predict(X_test) 
        y_scores = clf.predict_proba(X_test)

        score_p = metric_list(y_test,y_preds,y_scores, len(y.iloc[:,0].unique()))
        score[name_list[i]]= score_p
    
    df = pd.DataFrame.from_dict(score, orient='index', columns=['accuracy', 'precision', 'recall', 'f1','Macro-Averaged AUPRC',
                                                            'F_measure Beta=10','Modified mcc','MCC metirc','Gmean'])
    clf_score_dict[clf_name] = df
    
    #print(clf_score_dict[clf_name])

100%|██████████| 6/6 [04:17<00:00, 42.93s/it]


In [70]:
clf_score_dict['AdaBoost_SAMME']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.888000,0.857166,0.888000,0.868756,0.683584,0.885106,0.621345,0.805109,0.932203
Car Evaluation,0.849711,0.868063,0.849711,0.840497,0.544133,0.851347,0.594444,0.685234,0.040596
Contraceptive,0.576271,0.609650,0.576271,0.573188,0.610995,0.579154,0.351266,0.350759,0.213665
Dermatology,0.878378,0.905405,0.878378,0.868903,0.846271,0.880769,0.837876,0.848979,0.200000
Glass,0.627907,0.613999,0.627907,0.613924,0.602452,0.626617,0.499431,0.513841,0.000000
Hayes,0.500000,0.307692,0.500000,0.361111,0.833333,0.473118,0.441292,0.486427,1.000000
Heart,0.540984,0.481981,0.540984,0.507148,0.351246,0.535029,0.298713,0.316025,0.000000
New Thyroid,0.976744,0.977546,0.976744,0.976054,0.981400,0.976817,0.954171,0.955106,0.857143
Page,0.949772,0.947905,0.949772,0.948765,0.695361,0.949602,0.719292,0.737964,0.064709
Pen Based,0.676671,0.674940,0.676671,0.667727,0.666289,0.676513,0.640880,0.642441,0.010803


In [71]:
clf_score_dict['RUSBoostClassifier']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.880000,0.898701,0.880000,0.886372,0.721604,0.881668,0.718236,0.799587,0.832049
Car Evaluation,0.869942,0.885592,0.869942,0.875109,0.750568,0.871342,0.747724,0.739347,0.666629
Contraceptive,0.545763,0.579931,0.545763,0.554528,0.498861,0.548702,0.319427,0.322921,0.290381
Dermatology,0.972973,0.977887,0.972973,0.972394,0.983309,0.973418,0.961872,0.964759,0.750000
Glass,0.604651,0.616726,0.604651,0.607411,0.692422,0.605729,0.496916,0.495838,0.041667
Hayes,0.687500,0.495192,0.687500,0.568452,0.833333,0.664056,0.493461,0.567498,0.000000
Heart,0.491803,0.451190,0.491803,0.470310,0.324977,0.487811,0.227390,0.252337,0.000000
New Thyroid,0.883721,0.901339,0.883721,0.856187,0.974221,0.885294,0.750991,0.778656,0.285714
Page,0.745205,0.920426,0.745205,0.804781,0.769925,0.758329,0.458794,0.431549,0.606978
Pen Based,0.460209,0.472915,0.460209,0.404028,0.282988,0.461336,0.402798,0.414488,0.000000


In [72]:
clf_score_dict['GradientBoosting']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.816000,0.787066,0.816000,0.798012,0.697514,0.813282,0.492666,0.679347,0.800308
Car Evaluation,0.965318,0.973694,0.965318,0.967669,0.973750,0.966073,0.916324,0.928579,0.748663
Contraceptive,0.552542,0.554383,0.552542,0.549183,0.561179,0.552709,0.300376,0.300456,0.198232
Dermatology,0.986486,0.987838,0.986486,0.986392,1.000000,0.986609,0.980653,0.982152,0.875000
Glass,0.883721,0.893113,0.883721,0.877415,0.981689,0.884567,0.854021,0.852748,0.392857
Hayes,0.843750,0.865385,0.843750,0.845858,0.956776,0.845672,0.792569,0.764506,0.900000
Heart,0.491803,0.379372,0.491803,0.425061,0.376057,0.478901,0.191558,0.204533,0.000000
New Thyroid,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Page,0.715068,0.893834,0.715068,0.793868,0.370869,0.728310,0.264130,0.253892,0.000000
Pen Based,0.994088,0.994117,0.994088,0.994086,0.999692,0.994091,0.993448,0.993433,0.946399


In [73]:
clf_score_dict['XGBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.840000,0.829746,0.840000,0.833185,0.715320,0.839057,0.558187,0.722230,0.832974
Car Evaluation,0.979769,0.987636,0.979769,0.981765,0.943689,0.980479,0.945153,0.958866,1.000000
Contraceptive,0.586441,0.588074,0.586441,0.584939,0.582482,0.586589,0.353759,0.358250,0.247228
Dermatology,0.986486,0.987838,0.986486,0.986392,1.000000,0.986609,0.980653,0.982152,0.875000
Glass,0.837209,0.840091,0.837209,0.834931,0.911404,0.837470,0.786555,0.790912,0.261905
Hayes,0.812500,0.832418,0.812500,0.815020,0.952469,0.814271,0.734514,0.707742,0.750000
Heart,0.475410,0.407401,0.475410,0.438764,0.329632,0.468303,0.201437,0.218317,0.000000
New Thyroid,0.976744,0.977546,0.976744,0.976054,0.999582,0.976817,0.954171,0.955106,0.857143
Page,0.971689,0.972437,0.971689,0.971877,0.832299,0.971757,0.835902,0.858275,0.409673
Pen Based,0.990450,0.990513,0.990450,0.990450,0.999304,0.990456,0.989422,0.989393,0.911706


In [74]:
clf_score_dict['CatBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.880000,0.846267,0.880000,0.860520,0.723314,0.876823,0.608943,0.788488,0.915254
Car Evaluation,0.973988,0.980708,0.973988,0.975628,0.978974,0.974596,0.933166,0.946918,0.909091
Contraceptive,0.549153,0.547273,0.549153,0.543968,0.564997,0.548981,0.289413,0.297712,0.182649
Dermatology,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Glass,0.906977,0.914341,0.906977,0.907780,0.970088,0.907641,0.883432,0.882985,0.589286
Hayes,0.718750,0.767857,0.718750,0.716667,0.912292,0.722953,0.585374,0.561444,0.450000
Heart,0.475410,0.381148,0.475410,0.423059,0.354687,0.464956,0.181516,0.205881,0.000000
New Thyroid,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Page,0.975342,0.976794,0.975342,0.975681,0.887257,0.975474,0.858844,0.877837,0.623546
Pen Based,0.994543,0.994578,0.994543,0.994543,0.999725,0.994546,0.993937,0.993939,0.950758


In [75]:
clf_score_dict['AdaBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.944000,0.965778,0.944000,0.949436,0.697356,0.945939,0.882886,0.910854,0.881972
Car Evaluation,0.800578,0.801116,0.800578,0.778251,0.650552,0.800627,0.603533,0.573091,0.691205
Contraceptive,0.589831,0.596148,0.589831,0.585984,0.502358,0.590399,0.358602,0.365253,0.233743
Dermatology,0.851351,0.784459,0.851351,0.805219,0.803922,0.844802,0.778439,0.819744,0.000000
Glass,0.488372,0.481190,0.488372,0.425292,0.583236,0.487710,0.341356,0.326144,0.000000
Hayes,0.500000,0.307692,0.500000,0.361111,0.833333,0.473118,0.441292,0.486427,1.000000
Heart,0.508197,0.436690,0.508197,0.464272,0.318955,0.500743,0.245199,0.264758,0.000000
New Thyroid,0.720930,0.618605,0.720930,0.633926,0.653704,0.710250,0.331742,0.401677,0.000000
Page,0.738813,0.915992,0.738813,0.794022,0.676391,0.752037,0.517763,0.421373,0.000000
Pen Based,0.283765,0.243578,0.283765,0.170141,0.199340,0.279572,0.212055,0.256988,0.000000
